# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


/home/antoine/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lFeatures;
	}

	std::vector<std::any> get_classes(){
		std::vector<std::any> lClasses = { 0, 1, 2 };

		return lClasses;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { 
			"Score_0", "Score_1", "Score_2",
			"Proba_0", "Proba_1", "Proba_2",
			"LogProba_0", "LogProba_1", "LogProba_2",
			"Decision", "DecisionProba" };

		return lOutputs;
	}

	tTable compute_classification_scores(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {
		auto lClasses = get_classes();

		tTable lLogLikelihoodTable = tTable();

		lLogLikelihoodTable["Score"] = {
			-1.0986122886681098 + log_normal_cdf( Feature_0 , 5.005999999999999 , 0.12176400309550259 ) + log_normal_cdf( Feature_1 , 3.428000000000001 , 0.14081600309550263 ) + log_normal_cdf( Feature_2 , 1.4620000000000002 , 

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp__" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp__140280115077568.exe /tmp/sklearn2sql_cpp__140280115077568.cpp'
idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
0,,,,1.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,-41.14063451707327,-57.90531150297500,0,1.00000000000000
1,,,,1.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,-38.72865808550679,-56.71097802861089,0,1.00000000000000
2,,,,1.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,-41.37603426426448,-59.01694707418819,0,1.00000000000000
3,,,,1.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,-38.76127568122570,-56.48115330301085,0,1.00000000000000
4,,,,1.00000000000000,0.00000000000000,0.00000000000000,0.00000000000000,-42.23775062689118,-58.80806895839919,0,1.00000000000000
5,,,,0.99999999999999,0.00000000000001,0.00000000000000,-0.00000000000001,-31.83678286678461,-47.7

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,NaN,NaN,NaN,0.0,1.042537e-06,0.999999,-1.000000e+02,-13.773854,-1.042537e-06,2,0.999999
74,74,NaN,NaN,NaN,0.0,9.984608e-01,0.001539,-1.000015e+02,-0.001540,-6.476503e+00,1,0.998461
9,9,NaN,NaN,NaN,1.0,0.000000e+00,0.000000,0.000000e+00,-40.215873,-5.737607e+01,0,1.000000
88,88,NaN,NaN,NaN,0.0,9.997963e-01,0.000204,-1.000002e+02,-0.000204,-8.498714e+00,1,0.999796
25,25,NaN,NaN,NaN,1.0,0.000000e+00,0.000000,-0.000000e+00,-35.589051,-5.337152e+01,0,1.000000
5,5,NaN,NaN,NaN,1.0,1.000000e-14,0.000000,-1.000000e-14,-31.836783,-4.779038e+01,0,1.000000
48,48,NaN,NaN,NaN,1.0,0.000000e+00,0.000000,0.000000e+00,-40.476222,-5.653656e+01,0,1.000000
117,117,NaN,NaN,NaN,0.0,1.650000e-12,1.000000,-1.000000e+02,-27.132825,-1.650000e-12,2,1.000000
83,83,NaN,NaN,NaN,0.0,6.121598e-01,0.387840,-1.004908e+02,-0.490762,-9.471620e-01,1,0.612160
105,105,NaN,NaN,NaN,0.0,2.409800e-10,1.000000,-1.000000e+02,-22.146320,-2.409800e-10,2,1.000000


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,3.398457e-190,1.042537e-06,9.999990e-01,-4.362678e+02,-13.773854,-1.042537e-06,2
74,74,NaN,NaN,NaN,5.990232e-85,9.984608e-01,1.539184e-03,-1.939296e+02,-0.001540,-6.476503e+00,1
9,9,NaN,NaN,NaN,1.000000e+00,3.423483e-18,1.207506e-25,0.000000e+00,-40.215873,-5.737607e+01,0
88,88,NaN,NaN,NaN,6.730008e-74,9.997963e-01,2.037301e-04,-1.684847e+02,-0.000204,-8.498714e+00,1
25,25,NaN,NaN,NaN,1.000000e+00,3.498418e-16,6.622851e-24,-4.440892e-16,-35.589051,-5.337152e+01,0
5,5,NaN,NaN,NaN,1.000000e+00,1.490942e-14,1.757521e-21,-1.465494e-14,-31.836783,-4.779038e+01,0
48,48,NaN,NaN,NaN,1.000000e+00,2.638762e-18,2.795660e-25,0.000000e+00,-40.476222,-5.653656e+01,0
117,117,NaN,NaN,NaN,2.371497e-287,1.645749e-12,1.000000e+00,-6.599784e+02,-27.132825,-1.644906e-12,2
83,83,NaN,NaN,NaN,2.140697e-135,6.121598e-01,3.878402e-01,-3.100879e+02,-0.490762,-9.471620e-01,1
105,105,NaN,NaN,NaN,4.660355e-273,2.409770e-10,1.000000e+00,-6.270666e+02,-22.146320,-2.409770e-10,2


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
114,114,NaN,NaN,NaN,3.398457e-190,1.042537e-06,9.999990e-01,-4.362678e+02,-13.773854,-1.042537e-06,...,NaN,NaN,0.0,1.042537e-06,0.999999,-1.000000e+02,-13.773854,-1.042537e-06,2,0.999999
74,74,NaN,NaN,NaN,5.990232e-85,9.984608e-01,1.539184e-03,-1.939296e+02,-0.001540,-6.476503e+00,...,NaN,NaN,0.0,9.984608e-01,0.001539,-1.000015e+02,-0.001540,-6.476503e+00,1,0.998461
9,9,NaN,NaN,NaN,1.000000e+00,3.423483e-18,1.207506e-25,0.000000e+00,-40.215873,-5.737607e+01,...,NaN,NaN,1.0,0.000000e+00,0.000000,0.000000e+00,-40.215873,-5.737607e+01,0,1.000000
88,88,NaN,NaN,NaN,6.730008e-74,9.997963e-01,2.037301e-04,-1.684847e+02,-0.000204,-8.498714e+00,...,NaN,NaN,0.0,9.997963e-01,0.000204,-1.000002e+02,-0.000204,-8.498714e+00,1,0.999796
25,25,NaN,NaN,NaN,1.000000e+00,3.498418e-16,6.622851e-24,-4.440892e-16,-35.589051,-5.337152e+01,...,NaN,NaN,1.0,0.000000e+00,0.000000,-0.000000e+00,-35.589051,-5.337152e+01,0,1.000000
5,5,NaN,NaN,NaN,1.000000e+00,1.490942e-14,1.757521e-21,-1.465494e-14,-31.836783,-4.779038e+01,...,NaN,NaN,1.0,1.000000e-14,0.000000,-1.000000e-14,-31.836783,-4.779038e+01,0,1.000000
48,48,NaN,NaN,NaN,1.000000e+00,2.638762e-18,2.795660e-25,0.000000e+00,-40.476222,-5.653656e+01,...,NaN,NaN,1.0,0.000000e+00,0.000000,0.000000e+00,-40.476222,-5.653656e+01,0,1.000000
117,117,NaN,NaN,NaN,2.371497e-287,1.645749e-12,1.000000e+00,-6.599784e+02,-27.132825,-1.644906e-12,...,NaN,NaN,0.0,1.650000e-12,1.000000,-1.000000e+02,-27.132825,-1.650000e-12,2,1.000000
83,83,NaN,NaN,NaN,2.140697e-135,6.121598e-01,3.878402e-01,-3.100879e+02,-0.490762,-9.471620e-01,...,NaN,NaN,0.0,6.121598e-01,0.387840,-1.004908e+02,-0.490762,-9.471620e-01,1,0.612160
105,105,NaN,NaN,NaN,4.660355e-273,2.409770e-10,1.000000e+00,-6.270666e+02,-22.146320,-2.409770e-10,...,NaN,NaN,0.0,2.409800e-10,1.000000,-1.000000e+02,-22.146320,-2.409800e-10,2,1.000000


In [14]:
condition = (cpp_skl_join.Decision_cpp != cpp_skl_join.Decision_skl)
cpp_skl_join[condition]


,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
